In [19]:
import chromadb
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('KBLab/sentence-bert-swedish-cased')

chroma_client = chromadb.Client()

model = SentenceTransformer('KBLab/sentence-bert-swedish-cased')

In [9]:

sentences = ["hej"]

embeddings = model.encode(sentences)
print(embeddings)

[[ 1.81658342e-02  4.34152484e-02 -2.66788695e-02 -8.75618681e-02
   6.39092550e-02  1.42078949e-02  2.72023794e-03  3.03221177e-02
   1.38485236e-02 -1.25965755e-03 -3.60596031e-02 -4.47936654e-02
  -2.86959112e-03  4.64628078e-02  6.22472577e-02 -1.16787128e-01
   5.61639518e-02 -3.94901866e-03 -5.92752583e-02 -3.99692915e-02
  -3.27583216e-02  1.15990033e-02  4.88049984e-02  9.31422710e-02
   5.08920290e-02  1.66519750e-02  4.68375869e-02 -1.18874675e-02
  -2.74441461e-03  4.29601483e-02 -2.94235647e-02  6.37790887e-03
   8.82829204e-02  9.65920091e-03  4.37114659e-06 -1.05105517e-02
   1.32070659e-02  1.89248566e-02 -3.82460281e-02 -1.59527015e-04
   8.14241171e-03 -1.09644281e-02 -2.77733449e-02  5.41449971e-02
  -1.35887880e-04 -8.02094862e-02  4.25816663e-02 -6.72948137e-02
  -5.02526574e-02  4.89236526e-02  9.95047297e-03 -2.87219435e-02
  -7.24360719e-02 -4.79719229e-02  1.10050164e-01  8.22978541e-02
  -2.21166369e-02  3.30094509e-02 -2.56007072e-02 -8.17078054e-02
   6.27804

In [20]:
# Step 1: Read the text file
file_path = 'web_scraping/scraped_texts/text_1.txt'  # Replace with your file path
with open(file_path, 'r', encoding='utf-8') as file:
    text = file.read()

# Step 3: Generate embeddings
embedding_text1 = model.encode(text)

In [36]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
#testa embeddingarna 
den_lika = "att vara nedstämd och ledsen, perioder i livet"
Den_olika = "att vara lite orolig"

lika = model.encode(den_lika)
olika = model.encode(Den_olika)

# Example embeddings
embedding1 = np.array([embedding_text1])  # Note the double brackets (2D array)
embedding2 = np.array([lika])  # Note the double brackets (2D array)
embedding3 = np.array([olika])  # Note the double brackets (2D array)


# Compute cosine similarity
sim_lika = cosine_similarity(embedding1, embedding2)
sim_olika = cosine_similarity(embedding1, embedding3)

print("Cosine lika:", sim_lika[0][0])
print("Cosine olika:", sim_olika[0][0])


Cosine lika: 0.77475893
Cosine olika: 0.24562748


In [44]:
collection = chroma_client.create_collection(name="hassano_test_lol")


In [49]:
#chroma databas     

collection.add(embeddings=[embedding1[0].tolist(),embedding2[0].tolist(),embedding3[0].tolist()],
               documents=[text,den_lika, Den_olika],
               ids=["id1", "id2", "id3"])




Insert of existing embedding ID: id1
Insert of existing embedding ID: id2
Insert of existing embedding ID: id3
Add of existing embedding ID: id1
Add of existing embedding ID: id2
Add of existing embedding ID: id3


In [66]:
query = model.encode("nedstämdhet")

query_formatting = query.tolist()





In [60]:
print(query.tolist())

[0.04021087661385536, 0.027721494436264038, -0.05679550766944885, 0.007413405925035477, -0.03812437504529953, -0.015297596342861652, 0.0017580941785126925, 0.011278903111815453, 0.009184290654957294, 0.016612298786640167, -0.008416948840022087, -0.04281968995928764, -0.007936416193842888, -0.032114412635564804, -0.026282494887709618, 0.05369953438639641, -0.03336784243583679, 0.07840139418840408, 0.06729786098003387, -0.06427893787622452, 0.008867244236171246, 0.049978189170360565, 0.028262987732887268, -0.024498295038938522, -0.018606677651405334, -0.016043465584516525, 0.08697281777858734, 0.05061298608779907, -0.059528570622205734, -0.015032934956252575, 0.07772880792617798, -0.04186971113085747, -0.0669967532157898, 0.001920087612234056, 4.881757377006579e-06, 0.06587429344654083, -0.010452407412230968, 0.006783861666917801, -0.041029464453458786, 0.029060542583465576, 0.006224276497960091, -0.05332980304956436, -0.08372558653354645, 0.07585793733596802, -0.017231082543730736, 0.00

In [67]:
#query to chroma

collection.query(
    query_embeddings=[query_formatting],
    n_results=3,
)

{'ids': [['id2', 'id1', 'id3']],
 'distances': [[2.025731086730957, 2.6168675422668457, 3.2457845211029053]],
 'metadatas': [[None, None, None]],
 'embeddings': None,
 'documents': [['att vara nedstämd och ledsen, perioder i livet',
   'Till sidans huvudinnehåll\nBesök 1177.se, logga in för personliga e-tjänster eller ring 1177 för sjukvårdsrådgivning.\nLogga inpå 1177.se\nHitta vårdpå 1177.se\nSökpå 1177.se\nMeny\nLiv & hälsa\nBarn & gravid\nOlyckor & skador\nSjukdomar & besvär\nUndersökning & behandling\nSå fungerar vården\nStart\nLiv & hälsa\nPsykisk hälsa\nNedstämdhet\nNedstämdhet\nAtt känna sig nedstämd, håglös eller ledsen är en naturlig del av livet. Ibland förstår man varför man är nedstämd, ibland inte. Nedstämdhet går nästan alltid över av sig själv, men det kan ta tid. Det finns ofta saker du kan göra själv för att må bättre. Ibland kan du behöva hjälp.\nDen här artikeln handlar om nedstämdhet hos vuxna. Gäller\xa0detbarn som mår psykiskt dåligtkan du läsa mer här. Du som är